# Probe Designer


## Environment


In [1]:
# basci env
import os
import sys
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# # get gene data from ncbi
# from Bio import Entrez

# # blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# add package to sys var
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append("../lib")

# dir
workdir = './dataset/2023.12.15_Sindy_Isoforms/'
os.makedirs(workdir, exist_ok=True)

current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)

output = os.path.join(workdir, 'results', formatted_time)
pre_binding_dir = os.path.join(output, "pre_binding")
os.makedirs(output, exist_ok=True)

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total.fasta"

# tmp file
gene_name_list_file = "gene_list.xlsx"
pre_binding_num_file = "pre_binding_num.json"
blast_results_file = "blast_results.xml"

In [7]:
organism = 'mouse'
gene_info = pd.read_excel(os.path.join(workdir, gene_name_list_file), sheet_name='Sheet1')
# gene_list = list(gene_info['gene_name'].unique())
id_list = list(gene_info['ensembl_id'])[:1]
isoform_list = list(gene_info['isoform'])[:1]

sequences_of_all = dict()

## Get seq from ensembl dataset

In [8]:
from lib.database_interaction import ensembl_id_to_seqs
import time

skip = 0
trial = 0

for i in range(1):
    if trial > 3:
        skip += 1
        trial = 0
    try:
        tmp_isoform_list = isoform_list[skip:]
        tmp_id_list = id_list[len(sequences_of_all)+skip:]
        for i in range(len(tmp_isoform_list)):
            isoform = tmp_isoform_list[i]
            id = tmp_id_list[i]
            sequences = ensembl_id_to_seqs(gene=isoform, gene_id=id, seq_type='cds')
            for desc, sequence in sequences.items():
                sequences_of_all[desc] = sequence
        skip += 1
    except: 
        trial += 1
        time.sleep(5)

Gene:	Gm16024-201: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [11]:
id_list

['ENSMUST00000128841.1']

In [13]:
import requests
transcripts_url = f"http://rest.ensembl.org/overlap/id/{'ENSMUST00000128841'}?feature=transcript;content-type=application/json"
transcripts = requests.get(url=transcripts_url).json()
transcripts

[{'transcript_id': 'ENSMUST00000105608',
  'transcript_support_level': '5 (assigned to previous version 8)',
  'biotype': 'protein_coding',
  'logic_name': 'ensembl_havana_transcript_mus_musculus',
  'seq_region_name': '4',
  'is_canonical': 1,
  'assembly_name': 'GRCm39',
  'description': None,
  'ccdsid': 'CCDS38995.1',
  'source': 'ensembl_havana',
  'external_name': 'Slc35e2-202',
  'end': 155707797,
  'feature_type': 'transcript',
  'id': 'ENSMUST00000105608',
  'strand': 1,
  'start': 155685873,
  'version': 9,
  'Parent': 'ENSMUSG00000042202',
  'tag': 'basic'},
 {'start': 155686068,
  'strand': 1,
  'id': 'ENSMUST00000043829',
  'tag': 'basic',
  'version': 11,
  'Parent': 'ENSMUSG00000042202',
  'ccdsid': 'CCDS38995.1',
  'assembly_name': 'GRCm39',
  'description': None,
  'feature_type': 'transcript',
  'end': 155704547,
  'external_name': 'Slc35e2-201',
  'source': 'havana',
  'transcript_support_level': '1 (assigned to previous version 10)',
  'transcript_id': 'ENSMUST00000

In [10]:
sequences

{}

In [6]:
sequences

{}

In [5]:
len(sequences_of_all)

0

In [ ]:
import json
with open(os.path.join(output, 'sequences.json'), "w") as json_file:
    json.dump(sequences_of_all, json_file)

## Binding site Searcher


In [27]:
from lib.search_binding import step_by_step, find_max_min_difference_fixed_length_subsequence, seq_minus

# Initiation of array
binding_site_FOIs = [
    "accession",
    "gene_name",
    "mol_type",
    "organism",
    "pos_on_seq",
    "binding",
    "Tm_l",
    "Tm_r",
    "wanted",
]
align_FOIs = ["align_num", "align_accession", "align_descrip", "plus/minus"]
FOI = pd.DataFrame(columns=binding_site_FOIs + align_FOIs)

# Search binding sites on mRNA sequence
file_out_dir = pre_binding_dir
try:
    os.mkdir(file_out_dir)
except:
    pass

pre_binding_num = {}

# initialization of file
with open(os.path.join(file_out_dir, total_pre_binding_file_name), "w") as handle:
    handle.write("")

for desc, seq in sequences_of_all.items():
    id, gene_name, mol_type = desc.split('|')
    minus_seq = seq_minus(seq)
    
    Tm_l, Tm_r, selected_substrings, pos_on_seq = step_by_step(
        minus_seq,
        BDS_len=40,
        BDS_num=50,
        min_gap=1,
        better_gap=40,
        gene=gene_name,
        G_min=0.25,
        G_max=0.7,
        G_consecutive=5,
        Tm_low=50,
        Tm_high=65,
    )
    
    record_list = []
    for i, pre_binding_tmp in enumerate(selected_substrings):
        record_list.append(
            SeqRecord(
                Seq(pre_binding_tmp),
                id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type]),
            )
        )

    # add information about binding sites to FOI
    add = pd.DataFrame(
        {
            "accession": [id] * len(selected_substrings),
            "gene_name": [gene_name] * len(selected_substrings),
            "mol_type": [mol_type] * len(selected_substrings),
            "organism": [organism] * len(selected_substrings),
            "binding": selected_substrings,
            "Tm_l": Tm_l,
            "Tm_r": Tm_r,
            "pos_on_seq": pos_on_seq,
        }
    )
    FOI = pd.concat([FOI, add], ignore_index=True)

    file_out = os.path.join(file_out_dir, gene_name + pre_binding_file_suffix)
    
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
    with open(file_out_dir + total_pre_binding_file_name, "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(selected_substrings)

with open(os.path.join(output, pre_binding_num_file), "w") as f:
    json.dump(pre_binding_num, f)

position_searching_CDX2-201: 100%|██████████| 714/714 [00:00<00:00, 23093.97it/s]
C:\Users\Mingchuan\AppData\Local\Temp\ipykernel_41780\1689718117.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  FOI = pd.concat([FOI, add], ignore_index=True)


Gene CDX2-201: 	condition too harsh, loose to get better results
[0, 6, 12, 18, 24, 30, 36, 42, 51, 59, 69, 75, 149, 158, 164, 170, 178, 184, 192, 207, 232, 282, 288, 294, 301, 309, 315, 321, 329, 343, 349, 363, 411, 417, 423, 429, 436, 449, 456, 469, 475, 481, 487, 497, 503, 512, 521, 532, 541, 547, 604, 612, 654, 665, 683, 703]


position_searching_GATA3-202: 100%|██████████| 1029/1029 [00:00<00:00, 23686.62it/s]


Gene GATA3-202: 	condition too harsh, loose to get better results
[8, 28, 101, 155, 167, 179, 191, 203, 215, 227, 239, 251, 264, 276, 288, 300, 320, 332, 344, 357, 371, 383, 404, 416, 497, 509, 521, 533, 551, 563, 575, 587, 599, 611, 624, 636, 682, 694, 706, 718, 730, 743, 800, 819, 831, 898, 910, 925, 937, 949, 961, 973, 985, 997, 1009]


position_searching_YAP1-210: 100%|██████████| 1183/1183 [00:00<00:00, 26359.09it/s]


...

Gene PRKD1-202: 	condition too harsh, loose to get better results
[169, 174, 179, 184, 313, 786, 791, 796, 801, 809, 814, 819, 824, 829, 834, 839, 844, 892, 916, 1127, 1132, 1137, 1142, 1147, 1215, 1270, 1275, 1280, 1285, 1290, 1295, 1300, 1487, 1493, 1501, 1582, 1587, 1593, 1598, 1603, 1763, 1771, 1781, 1786, 1803, 1808, 1823, 1828, 1833, 1846, 1851, 1871, 2109, 2130]



position_searching_TEAD1-206: 100%|██████████| 985/985 [00:00<00:00, 35274.24it/s]


Gene TEAD1-206: 	condition too harsh, loose to get better results
[410, 424, 430, 441, 447, 485, 490, 494, 498, 502, 506, 510, 514, 518, 522, 526, 530, 535, 539, 543, 548, 587, 591, 595, 611, 615, 631, 635, 639, 643, 647, 654, 671, 691, 696, 703, 707, 711, 723, 727, 731, 738, 742, 746, 750, 754, 758, 766, 770, 774, 778, 790, 794, 886, 977, 984]


position_searching_NR2F2-201: 100%|██████████| 957/957 [00:00<00:00, 20860.03it/s]


Gene NR2F2-201: 	condition too harsh, loose to get better results
[33, 45, 109, 134, 146, 160, 172, 185, 197, 215, 235, 247, 259, 287, 307, 319, 344, 358, 370, 382, 394, 406, 419, 431, 443, 488, 500, 512, 524, 536, 580, 592, 604, 623, 680, 692, 704, 716, 728, 740, 752, 764, 776, 794, 806, 818, 830, 842, 868, 880]


## Blast and extract blast results

In [ ]:
# with open(file_out_dir + total_pre_binding_file_name, "r") as f:
#     fasta_string = f.read()
# txid = [2697049]  # organism

# # Submit BLAST search and get handle object
# handle = NCBIWWW.qblast(
#     program="blastn",
#     megablast="yes",
#     database="refseq_rna",
#     sequence=fasta_string,
#     url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
#     format_object="Alignment",
#     format_type="Xml",
# )

# # read handle object and save to a file
# with open(os.path.join(os.path.join(output, blast_results_file)), "w") as f:
#     f.write(handle.read())

In [45]:
# Extract interested information from blast_results
from Bio.Blast import NCBIXML


align_num = []
# read the id/plus-minus part/align_num
with open(os.path.join(output, blast_results_file), "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        FOI.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        FOI.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try:
            FOI.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except:
            FOI.loc[loca, "plus/minus"] = "NAN"

        loca += 1

FOI["align_num"] = align_num

## Select wanted binding site


In [46]:
FOI["wanted"] = [True] * len(FOI)

In [47]:
# sieve for the suitable binding site
gene_name_list = [_.upper() for _ in gene_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(FOI)):
    # check gene_name
    gene_name = FOI.loc[i, "gene_name"]
    if gene_name.split('-')[0].upper() not in gene_name_list:
        FOI.loc[i, "wanted"] = False
    else:
        try:
            gene_name_list_out.remove(gene_name)
        except:
            pass

    # check DNA or mRNA type
    if FOI.loc[i, "wanted"] == True:
        if FOI.loc[i, "mol_type"] != "protein_coding":
            FOI.loc[i, "wanted"] = False
            print(FOI.loc[i, "mol_type"])

    # check gene_organism name
    if FOI.loc[i, "wanted"] == True:
        spe_ori, gene_ori = FOI.loc[i, "organism"], FOI.loc[i, "gene_name"].split('-')[0]
        descrip = FOI.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                FOI.loc[i, "wanted"] = False
                break

    # check plus/minus
    if FOI.loc[i, "wanted"] == True:
        if pd.isnull(FOI.loc[i, "plus/minus"]):
            FOI.loc[i, "wanted"] = False
        else:
            pm_list = FOI.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                FOI.loc[i, "wanted"] = False

# write the whole information of interest to a excel file in tmp dir
FOI.to_excel(os.path.join(output, "probes_sieve.xlsx"))

out_tmp = FOI[FOI["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_of_True = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        length=3,
        min_gap=40,
        better_gap=80,
        gene=gene,
    )
    out_subset = out_tmp[out_tmp.gene_name == gene]
    out_subset = out_subset[out_subset["pos_on_seq"].isin(best_pos)]
    output_df = pd.concat([output_df, out_subset])

# write the output to a xlsx file
output_df.to_excel(os.path.join(output, "probes_wanted.xlsx"))

Gene FABP5-201: 	Not enough pos for 3 binding sites.
Gene FABP5-201: 	condition too harsh, loose to get better results
[143, 144]
